In [6]:
import numpy as np,torch
from torch.utils.data import DataLoader
from pathlib import Path
from translation_machine import collate_fn_mod
from translation_machine import dataset_mod,sentence_mod


In [7]:
import argparse
from pathlib import Path

batch_size = 32
d_model = 512
early_stopping_activated = False
early_stop_steps_per_half_clr_cycle = 3
nb_epochs = 60

limit_length= None
use_splitting = True
max_length_from_file = False


base_lr = 10**(-6)
max_lr = 0.0005
momentum = 0.9
half_period_cycle = 5
gamma = 0.8

load_from_backup = True
restore_optimizer = False

simple_hyp_params = argparse.Namespace(batch_size= batch_size,d_model = d_model,
early_stopping_activated = early_stopping_activated,
early_stop_steps_per_half_clr_cycle = early_stop_steps_per_half_clr_cycle,
nb_epochs = nb_epochs)

dset_truncation =argparse.Namespace(limit_length= limit_length,
                                    use_splitting = use_splitting,
                                    max_length_from_file = max_length_from_file)


opt_params = argparse.Namespace(base_lr = base_lr,
                                max_lr = max_lr,
                                momentum = momentum,
                                half_period_cycle = half_period_cycle,
                               gamma=gamma)

train_state_control = argparse.Namespace(load_from_backup = load_from_backup,
restore_optimizer = restore_optimizer
)


path_to_root = Path("../../")

paths = argparse.Namespace(path_dataset = "data/french_english_dataset/fra.txt",
path_language_info = "models/language_info.pth",
path_dataset_splitting = "dataset_splitting",
path_model_and_dependencies = f"models/sequence_translator_transformer_new.pth"
)

for key,path in paths.__dict__.items():
    paths.__dict__[key] = str(path_to_root.joinpath(path))



In [8]:
from ploomber_engine.ipython import PloomberClient
from ploomber import DAG
from pathlib import Path
from ploomber.products import File

# initialize client
client = PloomberClient.from_path(Path("./trainings/training_setup.ipynb"),cwd="./trainings/")
from argparse import Namespace

from translation_machine.models import transformer_mod
from translation_machine import sentence_mod

initial_namespace = argparse.Namespace(**{key:globals()[key] for key in ["simple_hyp_params","dset_truncation",
                                                  "opt_params","train_state_control",
                                                  "paths"]})
train_setup = client.get_namespace(initial_namespace.__dict__)
for key,val in train_setup.items():
        globals()[key] = val

Executing cell: 14: 100%|███████████████████████| 22/22 [01:21<00:00,  3.70s/it]


In [10]:
from translation_machine.models import transformer_mod

model_inputs = back_up_train_state["model_inputs"]

model = transformer_mod.TransformerForSeq2Seq(**model_inputs)

model.load_state_dict(back_up_train_state["model_params"])
model = model.eval()
model = model.to("cuda")
model.training

False

In [11]:
from translation_machine import translator_mod

translator = translator_mod.Translator(model)

In [12]:
translator.model = translator.model.eval()

In [ ]:

for english_sentence,french_sentence in val_dataset:
    translation = translator(english_sentence,limit_sentence=10)
    print(f"input sentence :  {english_sentence}")
    print(f"target sentence  : {french_sentence}")
    print(f"predicted sentence {translation}")
    print("\n")

/root/miniconda/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


input sentence :  That 's your half .
target sentence from ground truth : <sos> C' est ta moitié . <eos>
predicted sentence <sos> C ' est ton vous . <eos>


input sentence :  I scream a lot .
target sentence from ground truth : <sos> Je crie beaucoup . <eos>
predicted sentence <sos> J ' ai beaucoup beaucoup parlé . <eos>


input sentence :  Fire !
target sentence from ground truth : <sos> Au feu ! <eos>
predicted sentence <sos> Faites un peu de nouveau ! <eos>


input sentence :  The apple is red .
target sentence from ground truth : <sos> La pomme est rouge . <eos>
predicted sentence <sos> La lumière est rouge . <eos>


input sentence :  I started that .
target sentence from ground truth : <sos> C' est moi qui ai commencé . <eos>
predicted sentence <sos> J ' ai commencé ce sac . <eos>


input sentence :  Go on .
target sentence from ground truth : <sos> Continuez . <eos>
predicted sentence <sos> Va te faire bien . <eos>


input sentence :  The book is mine .
target sentence from groun